| SOURCE_TABLE_NAME | SOURCE_COLUMN_NAME | REPORTING_TABLE_NAME | REPORTING_COLUMN_NAME | TRANSFORMATION RULE | CONDITIONS |
| --- | --- |--- | --- |--- |--- |
| silver.daily_pricing_silver	| state_name	|***REPORTING_DIM_STATE_GOLD***	|***STATE_NAME***	| Select Unique state_name Values | Identify New/Changed Records From the Source Table . Use ***lakehouse_updated_date Column*** in source table to Identify New/Changed Records|
| DERIVED	| DERIVED	|***REPORTING_DIM_STATE_GOLD***	| ***STATE_ID***	| Generate Running Sequnece ID For Each Unique state_name Values | 1. Make Sure No Duplicate State_Name Values Loaded 2.  When loading new State_Name values in subsquent run , STATE_ID values need to be generated on top of existing MAX(STATE_ID)   |
| DERIVED	| DERIVED	|***REPORTING_DIM_STATE_GOLD***	| ***lakehouse_inserted_date***	| Load current_timestamp() | |
| DERIVED	| DERIVED	|***REPORTING_DIM_STATE_GOLD***	| ***lakehouse_updated_date***	| Load current_timestamp() | |

In [0]:
create or replace table `pricing-analytics`.silver.reporting_dim_state_stage_1
as
select distinct state_name from `pricing-analytics`.silver.daily_pricing_silver
where LAKEHOUSE_UPDATED_DATE > (select nvl(max(processed_table_datetime),'1900-01-01') from `pricing-analytics`.processrunlogs.deltalakehouse_process_runs where process_name='reportingDimensionsTablesLoad' and process_status='Completed')

In [0]:
create or replace table `pricing-analytics`.silver.reporting_dim_state_stage_2
as
select rdss1.state_name
, row_number() over (order by rdss1.state_name) + msi.max_state_id as state_id
, current_timestamp() as lakehouse_inserted_date
, current_timestamp() as lakehouse_updated_date
from `pricing-analytics`.silver.reporting_dim_state_stage_1 rdss1
left join `pricing-analytics`.gold.reporting_dim_state_gold rdsg
on rdss1.state_name=rdsg.state_name
cross join (select nvl(max(state_id),0) as max_state_id from `pricing-analytics`.gold.reporting_dim_state_gold) msi
where rdsg.state_name is null
;

In [0]:
insert into `pricing-analytics`.gold.reporting_dim_state_gold
select * from `pricing-analytics`.silver.reporting_dim_state_stage_2;